In [4]:
import numpy as np

In [3]:
# FUNCTION TO FIND ISOSURFACE
# PROVIDED WITH 3D FIELD AND TARGET SURFACE VALUE
# Can work with fields that are monotonically increasing or decreasing, specified in 'direction'
# The 'search' for the surface is done along the first dimesnion of the field array
def find_isosurface(field,target,direction):
    # Get the dimensions of the 3D field
    (nz,ny,nx)=field.shape
    # Pre-allocate output arrays based on these dimensions
    above = np.empty((nx,ny,)) # The index of the point below the surface
    above.fill(np.nan)
    below = np.empty((nx,ny,)) # The index of the point above the surface
    below.fill(np.nan)
    ratio = np.empty((nx,ny,)) # The ratio of the difference between the target and the field value at 'above' and the total difference between the 'above' and 'below' field values
    ratio.fill(np.nan)
    
    for i in range(0,nx):
        for j in range(0,ny):
            # For each column, find where the field is less than and more than the target surface
            lessarray = np.where(field[:,j,i] - target < 0)[0]
            morearray = np.where(field[:,j,i] - target > 0)[0]
            if morearray.size==0 or lessarray.size==0: # The surface does not exist
                above[i,j]=np.nan
                below[i,j]=np.nan
                ratio[i,j]=np.nan
            else:
                if direction=='increasing':
                    above[i,j] = lessarray[-1] # If the variable increases with depth, take the last field value that is less than target
                    below[i,j] = morearray[0] # ... and the first value that is greater than the target
                if direction=='decreasing':
                    below[i,j] = lessarray[0] # If the variable decreases with depth, take the first field value that is less than target
                    above[i,j] = morearray[-1] # ... and the last value that is greater than the target
                
                # 'above' index should be smaller than 'below' index (if monotonic). If this is not the case, skip past this grid cell.
                if above[i,j]<below[i,j]:
                    # Determine how far target surface is from the 'less' surface
                    diffabove = abs(field[above[i,j],j,i]-sigT)
                    diffbelow = abs(field[below[i,j],j,i]-sigT)
                    ratio[i,j] = diffabove/np.sum([diffabove, diffbelow]) # Distance from 'above' surface as a ratio of total field difference
                else:
                    above[i,j]=np.nan
                    below[i,j]=np.nan
                    ratio[i,j]=np.nan
                    
    return above, below, ratio